In [1]:
def loadGraph(filename):
    with open(str(filename), "r") as graph_instance:
        chromatic_num = int(graph_instance.readline())
        V, E = list(map(int, graph_instance.readline().split(" ")))
        graph = [[] for _ in range(V)]
        for line in graph_instance:
            if line.startswith('e'):
                line = line.split(" ")
                del line[0]
                edge = list(map(int, line))
                if (edge[1]-1) not in graph[edge[0]-1]:
                    graph[edge[0] - 1].append(edge[1] - 1)
                    graph[edge[1] - 1].append(edge[0] - 1)
    return chromatic_num, graph

In [2]:
def printSolution(color):
    print("Solution: ", end = " ")
    for i in range(len(color)):
        print(str(color[i]), end=" ")
        
    print()

In [3]:
def verticesDegrees(graph):
    degrees = []
    V = len(graph)
    
    for i in range(V):
        degrees.append(len(graph[i]))
        
    return degrees

In [4]:
import random
import math

In [5]:
def isFeasible(solution, graph):
    for i in range(len(graph)):
        for j in graph[i]:
            if(solution[i] == solution[j]):
                return False
    return True

In [6]:
def repair(solution, graph, m_degree):
    V = len(graph)
    
    for i in range(len(graph)):
        neighbors = [index for index in graph[i]]
        used_colors = [solution[neighbor] for neighbor in neighbors]
        if(solution[i] in used_colors):  
            available_colors = [c for c in range(1, V+1) if c not in used_colors]
            solution[i] = random.choice(available_colors)

In [7]:
def initialize(graph, m_degree):
    solution = []
    V = len(graph)
    
    for i in range(V):
        solution.append(random.randrange(1, V+1))
    
    if not isFeasible(solution, graph):
        repair(solution, graph, m_degree)
        
    return solution

In [8]:
def randomChangeOperator(solution, graph, m_degree):
    V = len(solution)
    position = random.randrange(V)
    
    old_color = solution[position]
    new_color = random.randrange(1, m_degree+1)
    
    solution[position] = new_color
    if isFeasible(solution, graph):
        m_degree = m_degree - 1
        return position, old_color
   
    repairVertexColor(solution, position, old_color)
    
    return -1, old_color

In [9]:
def repairVertexColor(solution, position, old_color):
    solution[position] = old_color

In [10]:
def calcSolutionValue(solution):
    return len(set(solution))

In [11]:
def simulatedAnnealing(graph, iters, chromatic_num, m_degree):
    solution = initialize(graph, m_degree)
    curr_value = calcSolutionValue(solution)
    best_value = curr_value 
    
    for i in range(1, iters):
        if curr_value == chromatic_num:
            break
        
        position, old_color = randomChangeOperator(solution, graph, m_degree)

        if position<0: 
            continue
        
        new_value = calcSolutionValue(solution)
        
        if new_value < curr_value:
            curr_value = new_value
            if new_value < best_value:
                best_value = new_value
        else:
            p = 1.0 / i ** 0.5
            q = random.uniform(0, 1)  
            if p > q:
                curr_value = new_value
            else:
                repairVertexColor(solution, position, old_color)
    
    return best_value, solution, i

In [14]:
graph = []
chromatic_num, graph = loadGraph("brute.txt")

m_degree = max(verticesDegrees(graph))
num_of_colors, color, num_of_iters = simulatedAnnealing(graph, 10000, chromatic_num, m_degree)
printSolution(color)

print(f'Chromatic number: {chromatic_num}, Found chromatic number: {num_of_colors}')
print(f'Number of iters: {num_of_iters}')

Solution:  5 3 2 1 2 
Chromatic number: 4, Found chromatic number: 4
Number of iters: 7


In [24]:
graph_list = ['brute.txt', 'myciel3.col', 'myciel4.col', 'myciel5.col', 'myciel6.col', 'david.col', 'huck.col', 'jean.col']
import time
from termcolor import colored

for graph_file in graph_list:
            print()
            print(colored(graph_file, color="red"))
            for max_iters in [100, 1000, 10000, 100000]:
                graph = []
                chromatic_num, graph = loadGraph(graph_file)
                m_degree = max(verticesDegrees(graph))
                
                print(f'---------------------Max iters: {max_iters}---------------------')
                start = time.time()
                num_of_colors, color, num_of_iters = simulatedAnnealing(graph, max_iters, chromatic_num, m_degree)
                end = time.time()
                exec_time = end - start
                
                print('time: ', exec_time)
                printSolution(color)
                print(f'Chromatic number: {chromatic_num}, Found chromatic number: {num_of_colors}')
                print(f'Number of iters: {num_of_iters}')


brute.txt
---------------------Max iters: 100---------------------
time:  3.5762786865234375e-05
Solution:  2 4 5 1 5 
Chromatic number: 4, Found chromatic number: 4
Number of iters: 1
---------------------Max iters: 1000---------------------
time:  3.552436828613281e-05
Solution:  3 5 2 4 2 
Chromatic number: 4, Found chromatic number: 4
Number of iters: 4
---------------------Max iters: 10000---------------------
time:  1.9311904907226562e-05
Solution:  2 4 3 5 3 
Chromatic number: 4, Found chromatic number: 4
Number of iters: 1
---------------------Max iters: 100000---------------------
time:  5.53131103515625e-05
Solution:  5 3 1 4 1 
Chromatic number: 4, Found chromatic number: 4
Number of iters: 4

myciel3.col
---------------------Max iters: 100---------------------
time:  0.00066375732421875
Solution:  3 4 2 2 4 5 4 2 5 5 3 
Chromatic number: 4, Found chromatic number: 4
Number of iters: 88
---------------------Max iters: 1000---------------------
time:  0.00037932395935058594


time:  3.127251148223877
Solution:  13 35 48 46 43 15 3 48 11 43 52 37 48 13 35 15 52 15 37 26 11 26 4 48 4 33 26 11 25 35 37 35 15 2 48 43 33 2 43 35 11 13 25 4 33 16 33 22 16 3 33 43 26 46 51 13 22 16 47 48 22 25 22 35 13 2 52 33 3 43 48 52 2 13 
Chromatic number: 11, Found chromatic number: 15
Number of iters: 99999

jean.col
---------------------Max iters: 100---------------------
time:  0.003113269805908203
Solution:  15 14 51 75 35 10 77 38 23 63 79 36 43 77 26 23 1 23 22 22 56 23 36 64 23 19 11 22 15 59 43 33 79 8 75 4 61 10 15 32 49 29 75 29 31 6 11 40 35 70 6 15 32 15 16 43 14 15 4 34 19 14 77 64 21 21 24 79 38 59 4 34 64 16 65 49 60 31 44 35 
Chromatic number: 10, Found chromatic number: 38
Number of iters: 99
---------------------Max iters: 1000---------------------
time:  0.02998661994934082
Solution:  14 7 20 25 28 69 21 7 20 11 58 21 35 26 5 11 54 53 23 17 14 35 6 2 6 23 32 4 53 3 9 22 4 11 22 11 10 17 9 8 35 35 21 28 8 28 26 8 28 17 28 25 25 2 58 11 17 7 21 69 19 35 7 26